# Importing Labraries

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [4]:
df_movies = pd.read_csv("../processed_data/movies.csv")
df_movies.head(2)

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,title,vote_average,...,collection,genres_list,spoken_languages_list,production_companies_list,production_countries_list,release_year,return,release_month,release_day,directors
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,...,Toy Story Collection,"['Animation', 'Comedy', 'Family']",['en'],['Pixar Animation Studios'],['US'],1995,12.45,octubre,lunes,['John Lasseter']
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Jumanji,6.9,...,NaN,"['Adventure', 'Fantasy', 'Family']","['en', 'fr']","['TriStar Pictures', 'Teitler Film', 'Intersco...",['US'],1995,4.04,diciembre,viernes,['Joe Johnston']


In [4]:
test = df_movies.sample(frac=0.6, random_state=69)
test

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,tagline,title,...,collection,genres_list,spoken_languages_list,production_companies_list,production_countries_list,release_year,return,transformed_title,release_month,release_day
44605,0.0,16465,en,The Infernal Boiling Pot is a 1903 silent Fren...,0.262724,1903-10-17,0.0,1.0,NaN,The Infernal Caldron,...,NaN,"['Fantasy', 'Horror']",['xx'],['Star Film Company'],['FR'],1903,0.00,theinfernalcaldron,octubre,sábado
21407,0.0,154282,en,Abby is a sought after massage therapist and a...,2.976672,2013-01-19,35022.0,89.0,A self-improved comedy,Touchy Feely,...,NaN,['Drama'],['en'],['Lynn Shelton'],['US'],2013,0.00,touchyfeely,enero,sábado
10657,0.0,43899,en,Charles Dickens' classic tale of an orphaned b...,1.132759,1935-01-18,0.0,130.0,1935's most beloved motion picture!,David Copperfield,...,NaN,"['Adventure', 'Drama', 'Romance']",['en'],['Metro-Goldwyn-Mayer (MGM)'],['US'],1935,0.00,davidcopperfield,enero,viernes
1156,18500000.0,679,en,When Ripley's lifepod is found by a salvage cr...,21.761179,1986-07-18,183316455.0,137.0,This Time It's War,Aliens,...,NaN,"['Horror', 'Action', 'Thriller', 'Science Fict...",['en'],"['Twentieth Century Fox Film Corporation', 'SL...","['GB', 'US']",1986,9.91,aliens,julio,viernes
7015,10000000.0,24034,en,Spot (Lane) is a dog who can talk and read. Po...,0.491334,2004-01-16,0.0,74.0,A musical tail about one dog's dream of becomi...,Teacher's Pet,...,NaN,"['Animation', 'Drama', 'Family', 'Music']",['en'],"['Walt Disney Pictures', 'Walt Disney Televisi...",['US'],2004,0.00,teacherspet,enero,viernes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37779,0.0,161816,en,Ankur Goyal is a guileless 30-year-old waiter ...,0.003494,2003-08-14,0.0,96.0,NaN,Raghu Romeo,...,NaN,[],['hi'],[],[],2003,0.00,raghuromeo,agosto,jueves
27816,0.0,307946,en,"Laney is an attractive, intelligent suburban w...",4.680849,2015-10-23,0.0,85.0,"Love desperately, live recklessly.",I Smile Back,...,NaN,['Drama'],['en'],"['Egoli Tossell Film', 'Koppelman/Levien']",['US'],2015,0.00,ismileback,octubre,viernes
10820,18000000.0,9968,en,"To remedy his financial problems, a travel age...",12.283163,2005-10-27,0.0,100.0,"When you need somebody, anybody will do.",The Big White,...,NaN,"['Comedy', 'Crime', 'Drama']",['en'],"['Capitol Films', 'Ascendant Pictures', 'Risin...","['CA', 'DE', 'NZ', 'US']",2005,0.00,thebigwhite,octubre,jueves
42439,0.0,401104,en,A young woman dealing with anorexia meets an u...,9.128637,2017-01-22,0.0,107.0,NaN,To the Bone,...,NaN,['Drama'],['en'],"['Mockingbird Pictures', 'AMBI Group', 'Sparkh...",['US'],2017,0.00,tothebone,enero,domingo


In [5]:
tfidf = TfidfVectorizer(stop_words="english")

In [7]:
test["genres_list"].fillna("", inplace=True)
tfidf_matrix = tfidf.fit_transform(test["genres_list"])

In [8]:
cosine_sin = linear_kernel(tfidf_matrix, tfidf_matrix) 
#np.save("cosine_sim.npy", cosine_sin)

In [9]:
index = pd.Series(test.index, test["title"]).drop_duplicates()

In [10]:
def get_recommendations(title, cosine_sin = cosine_sin):
    idx = index[title]
    sim_scores = enumerate(cosine_sin[idx])
    sim_scores = sorted(sim_scores,key = lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]

    sim_index = [i[0]for i in sim_scores]
    print(df_movies["title"].iloc[sim_index])

get_recommendations('Aliens')

841                  Kansas City
1156                      Aliens
1619                     Flubber
1631    Other Voices Other Rooms
1763               Clockwatchers
2173                  The Player
2298       Young Sherlock Holmes
2411                   Westworld
2637                Light of Day
2774         Plunkett & MacLeane
Name: title, dtype: object


import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# DataFrame original
df_movies = pd.read_csv('movies.csv')

# Muestreo para entrenamiento
df_sample = df_movies.sample(n=100)  # Tomar una muestra de 100 películas

# Preprocesamiento de los datos
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_sample['overview'])

# Entrenamiento del modelo cosine_sim
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Objeto para el cual deseas obtener recomendaciones
new_object = "A new movie with a unique plot"

# Preprocesamiento del nuevo objeto
new_object_tfidf = tfidf.transform([new_object])

# Cálculo de similitud entre el nuevo objeto y las películas de la muestra
similarity_scores = cosine_similarity(new_object_tfidf, tfidf_matrix)

# Obtención de las recomendaciones basadas en la similitud
top_indices = similarity_scores.argsort()[0][-5:]  # Obtener los índices de las 5 películas más similares
recommendations = df_sample.iloc[top_indices]['title'].tolist()  # Obtener los títulos de las películas recomendadas

print("Recomendaciones:")
for movie in recommendations:
    print(movie)


In [15]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier

# Crear un DataFrame de ejemplo
data = {
    'title': ['Pelicula 1', 'Pelicula 2', 'Pelicula 3'],
    'budget': [1000000, 2000000, 1500000],
    "revenue": [10000000, 2030000, 500000],
    'duration': [120, 90, 105],
    'votes': [500, 1000, 800],
    'genre': ['Action', 'Comedy', 'Drama'],
    'synopsis': ['Una pelicula de accion emocionante', 'Una comedia divertida', 'Un drama conmovedor']
}

df = pd.DataFrame(data)

# Crear los vectores TF-IDF de la columna de sinopsis
tfidf = TfidfVectorizer(stop_words='english')
synopsis_tfidf = tfidf.fit_transform(df['synopsis'])

# Entrenar un modelo de regresión lineal para predecir el éxito financiero (recaudación)
regression_model = LinearRegression()
regression_model.fit(df[['budget', 'duration', 'votes']], df['revenue'])

# Entrenar un modelo de clasificación para predecir el género de la película
classification_model = KNeighborsClassifier(n_neighbors=3)
classification_model.fit(synopsis_tfidf, df['genre'])

# Función de recomendación que combina ambas predicciones
def recommend_movie(title):
    # Obtener la información de la película de interés
    movie = df[df['title'] == title]
    budget = movie['budget'].values[0]
    duration = movie['duration'].values[0]
    votes = movie['votes'].values[0]
    synopsis = movie['synopsis'].values[0]

    # Realizar las predicciones
    revenue_prediction = regression_model.predict([[budget, duration, votes]])
    genre_prediction = classification_model.predict(tfidf.transform([synopsis]))
    
    # Filtrar las películas recomendadas por género y recaudación
    recommended_movies = df[df['genre'] == genre_prediction[0]]
    recommended_movies = recommended_movies[recommended_movies['revenue'] >= revenue_prediction[0]]
    
    return recommended_movies['title']

# Ejemplo de recomendación
recommendations = recommend_movie('Pelicula 1')
print(recommendations)


0    Pelicula 1
Name: title, dtype: object


c:\Users\ASUS X515\Documents\Programacion_Data\proyectos\movie-recommendation-system\venv\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
